In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table", "view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [3]:
%%sql
SELECT *
FROM customer
LIMIT 5

Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


In [4]:
%%sql
WITH totalnumber AS
    (SELECT COUNT(*) total
     FROM invoice_line il
     LEFT JOIN invoice i ON i.invoice_id=il.invoice_id
     WHERE i.billing_country = "USA"
    )

SELECT
    g.name genre,
    COUNT(*) number,
    ROUND(CAST(COUNT(*) AS FLOAT) / totalnumber.total * 100,2) percentage
FROM invoice_line il, totalnumber
LEFT JOIN invoice i ON i.invoice_id=il.invoice_id
LEFT JOIN track t ON t.track_id=il.track_id
LEFT JOIN genre g ON g.genre_id=t.genre_id
WHERE i.billing_country="USA"
GROUP BY 1

Done.


genre,number,percentage
Alternative,35,3.33
Alternative & Punk,130,12.37
Blues,36,3.43
Classical,4,0.38
Easy Listening,13,1.24
Electronica/Dance,5,0.48
Heavy Metal,3,0.29
Hip Hop/Rap,20,1.9
Jazz,14,1.33
Latin,22,2.09


Rock genre sold more than 50% of all tracks in USA.

In [5]:
%%sql
SELECT
    ar.name artist,
    COUNT(il.invoice_id) number_of_tracks
FROM invoice_line il
LEFT JOIN track t ON t.track_id=il.track_id
LEFT JOIN genre g ON g.genre_id=t.genre_id
LEFT JOIN album al ON al.album_id=t.album_id
LEFT JOIN artist ar ON ar.artist_id=al.artist_id
WHERE g.name = "Rock"
GROUP BY 1
ORDER BY 2 DESC
LIMIT 3

Done.


artist,number_of_tracks
Queen,192
Jimi Hendrix,187
Nirvana,130


Top 3 artists with most tracks from Rock sold are...  
1. Queen  
2. Jimi Hendrix   
3. Nirvana  

In [6]:
%%sql
WITH employee_sales AS
(
    SELECT
        e.employee_id,
        e.hire_date,
        e.title,
        ROUND(SUM(i.total),2) total_sales
    FROM employee e
    LEFT JOIN customer c ON c.support_rep_id=e.employee_id
    LEFT JOIN invoice i ON i.customer_id=c.customer_id
    GROUP BY 1
    HAVING total_sales != 0
    ORDER BY 4 DESC
)

SELECT 
    e.first_name || " " || e.last_name Name,
    e.hire_date,
    es.total_sales
FROM employee e
INNER JOIN employee_sales es ON es.employee_id=e.employee_id

Done.


Name,hire_date,total_sales
Jane Peacock,2017-04-01 00:00:00,1731.51
Margaret Park,2017-05-03 00:00:00,1584.0
Steve Johnson,2017-10-17 00:00:00,1393.92


The total sales seems correlated with hire date where the earlier the employee started to work there, the greater total sales the employee made.

In [7]:
%%sql

SELECT
    c.customer_id,
    c.country,
    i.invoice_id,
    i.total
FROM customer c
INNER JOIN invoice i ON i.customer_id=c.customer_id
ORDER BY 1
LIMIT 5

Done.


customer_id,country,invoice_id,total
1,Brazil,16,8.91
1,Brazil,77,5.9399999999999995
1,Brazil,149,8.91
1,Brazil,153,13.86
1,Brazil,182,5.9399999999999995


In [8]:
%%sql
WITH

country_other_table AS
(
    SELECT
        country,
        COUNT(DISTINCT customer_id),
        CASE
            WHEN COUNT(DISTINCT customer_id)=1
                THEN 1
                ELSE 0
            END AS country_other
    FROM customer
    GROUP BY 1
),

country_2_table AS
(
    SELECT
        country,
        CASE
            WHEN country_other = 1
                THEN "Other"
                ELSE country
            END AS country2,
        country_other
    FROM country_other_table
),

all_cus AS
(
    SELECT
        c.customer_id,
        c2t.country2,
        i.invoice_id,
        i.total,
        c2t.country_other
    FROM customer c
    LEFT JOIN country_2_table c2t ON c2t.country=c.country
    LEFT JOIN invoice i ON i.customer_id=c.customer_id
)

SELECT
    country2 country,
    COUNT(DISTINCT customer_id) num_of_customers,
    ROUND(SUM(total),2) total_sales,
    ROUND(SUM(total)/COUNT(DISTINCT customer_id),2) avg_per_cus,
    ROUND(SUM(total)/COUNT(DISTINCT invoice_id),2) avg_per_ord
FROM all_cus
GROUP BY 1
ORDER BY country_other

Done.


country,num_of_customers,total_sales,avg_per_cus,avg_per_ord
Brazil,5,427.68,85.54,7.01
Canada,8,535.59,66.95,7.05
Czech Republic,2,273.24,136.62,9.11
France,5,389.07,77.81,7.78
Germany,4,334.62,83.65,8.16
India,2,183.15,91.57,8.72
Portugal,2,185.13,92.56,6.38
USA,13,1040.49,80.04,7.94
United Kingdom,3,245.52,81.84,8.77
Other,15,1094.94,73.0,7.45


In [9]:
%%sql

WITH
invoice_track AS
(
    SELECT 
        il.invoice_id,
        il.track_id,
        t.album_id
    FROM invoice_line il
    LEFT JOIN track t ON t.track_id=il.track_id    
)
SELECT
    it.invoice_id,
    CASE
        WHEN
            (SELECT track_id FROM invoice_line WHERE invoice_id=it.invoice_id
             EXCEPT
             SELECT track_id FROM track WHERE album_id=it.album_id) IS NULL
            AND
            (SELECT track_id FROM track WHERE album_id=it.album_id
             EXCEPT
             SELECT track_id FROM invoice_line WHERE invoice_id=it.invoice_id) IS NULL
            THEN 1
            ELSE 0
            END AS album_purchase
FROM invoice_track it
GROUP BY 1

Done.


invoice_id,album_purchase
1,1
2,0
3,0
4,0
5,1
6,0
7,0
8,0
9,0
10,0


In [10]:
%%sql

WITH
invoice_track AS
(
    SELECT 
        il.invoice_id,
        il.track_id,
        t.album_id
    FROM invoice_line il
    LEFT JOIN track t ON t.track_id=il.track_id    
),

invoice_album AS
(
    SELECT
        it.invoice_id,
        CASE
            WHEN
                (SELECT track_id FROM invoice_line WHERE invoice_id=it.invoice_id
                 EXCEPT
                 SELECT track_id FROM track WHERE album_id=it.album_id) IS NULL
                AND
                (SELECT track_id FROM track WHERE album_id=it.album_id
                 EXCEPT
                 SELECT track_id FROM invoice_line WHERE invoice_id=it.invoice_id) IS NULL
                THEN 1
                ELSE 0
                END AS album_purchase
    FROM invoice_track it
    GROUP BY 1
)

SELECT
    SUM(album_purchase) album_purchases,
    ROUND((SUM(album_purchase)/CAST(COUNT(*) AS FLOAT))*100,2) percentage
FROM invoice_album

Done.


album_purchases,percentage
114,18.57


Although the percentage is lower than 20% of which a invoice represents an album purchase, it does not necessarily say that the profit from selling albums is lower than individual tracks purchases. Further exploration needs to be carried out.